In [ ]:
#Extracting the features of interest from each soundtrack, and taking the mean of each.
#Each soundtrack is represented by six features.

import numpy as np
import pandas as pd
import os
import csv 

Y=[]
data=[]

genres=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
for genre in genres:
    directory='/Users/iuliia/Desktop/music/'+str(genre)+'/'
    files=os.listdir(directory)
    for audio in files:
        audiopath = directory+audio
        if audiopath=='/Users/iuliia/Desktop/music/blues/.DS_Store':
            pass
        else:
            y, sr = librosa.load(audiopath, mono=True, duration=30)
    
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            line=[np.mean(chroma_stft),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr),np.mean(mfcc), genre]
            
            file = open('music.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(line)

In [10]:
#SVC model.
data=pd.read_csv('music.csv')
data=pd.DataFrame(data)
data.columns=['chroma_stft', 'spec_cent','spec_bw', 'rolloff', 'zcr', 'mfcc', 'genre']
y=data['genre']
X=data.drop('genre', axis=1)

from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[12  4  0  6  0  6  8  0  0  0]
 [ 0 19  0  0  0  2  0  0  1  0]
 [17  2  3  1  4  4  0  5  3  0]
 [ 5  1  0  9  8  1  5  7  0  2]
 [ 2  0  0  3  7  0  8  8  2  0]
 [11  3  2  3  2  9  0  8  2  0]
 [ 1  0  0  2  0  0 28  0  0  0]
 [ 0  0  1  2  1  0  0 22  2  0]
 [ 8  1  1  1  3  0  3  7  5  0]
 [ 8  0  1  5  3  4  3  6  4  3]]
              precision    recall  f1-score   support

       blues       0.19      0.33      0.24        36
   classical       0.63      0.86      0.73        22
     country       0.38      0.08      0.13        39
       disco       0.28      0.24      0.26        38
      hiphop       0.25      0.23      0.24        30
        jazz       0.35      0.23      0.27        40
       metal       0.51      0.90      0.65        31
         pop       0.35      0.79      0.48        28
      reggae       0.26      0.17      0.21        29
        rock       0.60      0.08      0.14        37

    accuracy                           0.35       330
   macro avg       

In [11]:
#Logistic regression.
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
svclassifier = logisticRegr = LogisticRegression()
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 2  5  3  9  3  5  6  0  3  0]
 [ 0 20  0  0  0  1  0  0  1  0]
 [ 5  5  7  0  4  3  0  8  6  1]
 [ 0  1  0  7  5  1 10  8  6  0]
 [ 0  0  3  5  7  0  7  7  1  0]
 [ 2  5  6  2  0 10  0 10  5  0]
 [ 0  0  0  1  4  0 26  0  0  0]
 [ 1  0  2  0  1  0  0 24  0  0]
 [ 4  1  3  0  3  0  3  8  6  1]
 [ 4  1  3  5  4  3  6  7  4  0]]
              precision    recall  f1-score   support

       blues       0.11      0.06      0.07        36
   classical       0.53      0.91      0.67        22
     country       0.26      0.18      0.21        39
       disco       0.24      0.18      0.21        38
      hiphop       0.23      0.23      0.23        30
        jazz       0.43      0.25      0.32        40
       metal       0.45      0.84      0.58        31
         pop       0.33      0.86      0.48        28
      reggae       0.19      0.21      0.20        29
        rock       0.00      0.00      0.00        37

    accuracy                           0.33       330
   macro avg       

/Users/iuliia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/iuliia/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [4]:
#Fully-connected model.
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn import preprocessing
from keras import models
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras import layers

data=pd.read_csv('music.csv')
genre_list = data.iloc[:, -1]
encoder = preprocessing.LabelEncoder()  #converts genres names from strings to integers.
y = encoder.fit_transform(genre_list)
scaler = preprocessing.StandardScaler() #Normalizing the dataset
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=100)
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

Epoch 1/100
800/800 [==============================] - 0s 171us/step - loss: 2.2005 - accuracy: 0.2675
Epoch 2/100
800/800 [==============================] - 0s 24us/step - loss: 2.0275 - accuracy: 0.3150
Epoch 3/100
800/800 [==============================] - 0s 29us/step - loss: 1.8613 - accuracy: 0.3487
Epoch 4/100
800/800 [==============================] - 0s 25us/step - loss: 1.7298 - accuracy: 0.3862
Epoch 5/100
800/800 [==============================] - 0s 29us/step - loss: 1.6346 - accuracy: 0.4025
Epoch 6/100
800/800 [==============================] - 0s 25us/step - loss: 1.5641 - accuracy: 0.4250
Epoch 7/100
800/800 [==============================] - 0s 36us/step - loss: 1.5196 - accuracy: 0.4400
Epoch 8/100
800/800 [==============================] - 0s 25us/step - loss: 1.4830 - accuracy: 0.4512
Epoch 9/100
800/800 [==============================] - 0s 30us/step - loss: 1.4578 - accuracy: 0.4487
Epoch 10/100
800/800 [==============================] - 0s 25us/step - loss: 1.44

800/800 [==============================] - 0s 28us/step - loss: 0.9571 - accuracy: 0.6438
Epoch 81/100
800/800 [==============================] - 0s 26us/step - loss: 0.9453 - accuracy: 0.6513
Epoch 82/100
800/800 [==============================] - 0s 30us/step - loss: 0.9402 - accuracy: 0.6475
Epoch 83/100
800/800 [==============================] - 0s 26us/step - loss: 0.9461 - accuracy: 0.6475
Epoch 84/100
800/800 [==============================] - 0s 23us/step - loss: 0.9359 - accuracy: 0.6350
Epoch 85/100
800/800 [==============================] - 0s 25us/step - loss: 0.9368 - accuracy: 0.6513
Epoch 86/100
800/800 [==============================] - 0s 24us/step - loss: 0.9252 - accuracy: 0.6525
Epoch 87/100
800/800 [==============================] - 0s 27us/step - loss: 0.9189 - accuracy: 0.6637
Epoch 88/100
800/800 [==============================] - 0s 25us/step - loss: 0.9218 - accuracy: 0.6562
Epoch 89/100
800/800 [==============================] - 0s 24us/step - loss: 0.9159 - 

In [3]:
#Function to perform audio segmentation.
def segmentation(path, audio, segment_length):
    sound = AudioSegment.from_wav(path)
    seconds=np.arange(0, 30001, segment_length*1000).tolist()
    path=path[:-4] #cuts off '.wav'
    
    
    count=0
    for interval in range(1, len(seconds)):
        count+=1
        start, end = seconds[interval-1], seconds[interval]
        sound[start:end].export('/Users/iuliia/Desktop/segments1/'+str(audio[:-4])+'_'+str(count)+'.wav', format="wav") #Write the segmented file to the right directory.

In [283]:
#Extracting features from each segment and writing to a corresponding folder.
genres=['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

for genre in genres:
    directory='/Users/iuliia/Desktop/music/'+str(genre)+'/'
    files=os.listdir(directory)
    
    for audio in files:
        path = directory+audio
        if path=='/Users/iuliia/Desktop/music/blues/.DS_Store':
            pass
        
        elif path=='/Users/iuliia/Desktop/music/country/.DS_Store':
            pass
        else:
            
            segmented_directory='/Users/iuliia/Desktop/segments10/'
            
            #if os.path.isdir(segmented_directory)==False: #Check if directory exists.
             #   os.mkdir(segmented_directory) #If not, create a new folder
        
            os.chdir(segmented_directory) #Change the directory
            segmentation(path, audio, 1)